In [13]:
import duckdb
import xml.etree.ElementTree as ET
# Import classifier
from ipynb.fs.full.group_classifier import predict_batch
print("Import successful!!")

Import successful!!


In [14]:
 # Connect to sql database
con = duckdb.connect(database='../data/database/german-parliament', read_only=False)

In [24]:
def preprocess_speech(speech:tuple) -> tuple[int, list[str]]:
    # We only need the speech it to later identify the origin speech
    speech_id = test_speech[0]
    text = test_speech[5]
    # Parse xml from string will throw ParseError if not parseable
    root = ET.fromstring(text)
    # We only need paragraphs -> this filters unecessary information, such as comments or applause
    paragraphs = root.findall('p')
    paragraphs_text = []
    # Get text of paragraphs
    for p in paragraphs:
        # Use this to filter out speaker information
        if p.attrib.get("klasse") == "redner":
            continue
        else:
            p_text = p.text.strip()
            item = {"paragraphs":p_text}
            paragraphs_text.append(item) #get text and remove potential irrelevant whitespaces

    return (speech_id, paragraphs_text)
    

In [4]:
test_speech = con.sql("select * from speech where id like '%ID%' order by random() limit 1 ").fetchone()

In [25]:
paragraphs = [{'paragraphs': 'Frau Präsidentin! Meine sehr geehrten Damen und Herren Abgeordnete! Umwelt- und Klimaschutz zahlen sich aus\xa0– das hat heute Morgen unser Finanzminister Olaf Scholz sehr eindrucksvoll dargelegt. Das ist eine der zentralen Botschaften des Bundeshaushaltes für 2021, den die Bundesregierung diese Woche hier in den Deutschen Bundestag einbringt. Sie zahlen sich aus, weil sie gute, gesunde Lebensbedingungen sicherstellen, sie zahlen sich aus, weil sie Standortvorteile sind und Wettbewerbsfähigkeit von morgen sicherstellen, und sie zahlen sich aus, weil sie die Teilhabe aller Menschen in unserem Land sichern und damit auch den Zusammenhalt stärken.'}, {'paragraphs': 'Der Klimaschutz ist daher\xa0– und das ist genau richtig\xa0– auch nicht die alleinige Aufgabe der Umweltministerin. Ich habe immer wieder eingefordert, dass die gesamte Bundesregierung, also alle Ministerien, hier Verantwortung übernehmen. Und dieser Haushaltsentwurf zeigt, dass inzwischen alle Ministerien auf dem Weg sind, Klimaministerien zu werden. Das haben Sie gerade bei der Einbringung des Haushalts des Verkehrsministeriums gehört'}, {'paragraphs': '– ja, natürlich!\xa0–, welches dank Olaf Scholz verstärkt in Bus, in Bahn, in Elektromobilität investiert. Und das sehen Sie in den Haushaltsentwürfen des Wirtschafts-, des Landwirtschafts-, des Bauministeriums.'}, {'paragraphs': 'Die Bundesregierung wird in 2021 so viel wie nie zuvor in der Geschichte der Bundesrepublik in den Klimaschutz investieren.'}, {'paragraphs': 'Sehen Sie sich allein die Ausgaben für den Energie- und Klimafonds an: Sie werden mit knapp 27\xa0Milliarden Euro\xa0– nicht Millionen\xa0– mehr als verdreifacht. Der vorliegende Haushaltsentwurf ist damit die konsequente Fortsetzung des Weges, den die Bundesregierung mit dem Klimaschutzprogramm\xa02030 und dem Klimaschutzgesetz eingeschlagen hat. Wir bekennen uns zum Ziel der Klimaneutralität in 2050, und wir unterstützen Wirtschaft und Gesellschaft auf dem Weg dahin. Wir investieren ganz massiv in Forschung und Innovation. Wenn Sie sich allein mein Ministerium ansehen: Über den Energie- und Klimafonds sind Investitionen in die Dekarbonisierung der Industrie vorgesehen. Im Finanzplan bis 2024 stehen dafür rund 1,5\xa0Milliarden Euro zur Verfügung. Ich glaube, das ist sehr gut investiertes Geld.'}, {'paragraphs': 'Wir unterstützen den Markthochlauf von Grünem Wasserstoff\xa0– das ist wichtig, weil wir Klimaneutralität in der Stahlproduktion sowie in den Bereichen Chemie und Luftfahrt wollen; das eröffnet starke industriepolitische Perspektiven für Deutschland\xa0–, und wir setzen im Umwelt- und im Klimaschutz beim Kampf gegen den Artenverlust auf neue Technologien wie Digitalisierung und KI und fördern innovative Pilotprojekte.'}, {'paragraphs': 'Mir ist besonders wichtig, zu betonen: Wir schaffen neue Perspektiven in den vom Strukturwandel besonders betroffenen Regionen. Mit diesem Bundeshaushalt werden erstmals Mittel zur Strukturstärkung in den Kohleregionen eingestellt. Im Rahmen des Strukturstärkungsgesetzes ist das Bundesumweltministerium mit einer Vielzahl von Projekten an der Umsetzung von Fördermaßnahmen beteiligt. Hierzu zählen das Zentrum für Biodiversitätsmonitoring in Leipzig, das Kompetenzzentrum für PtX, das Kompetenzzentrum Elektromagnetische Felder sowie das Kompetenzzentrum Klimaschutz in energieintensiven Industrien, alle in der Lausitz beheimatet.'}, {'paragraphs': 'Diese Einrichtungen liegen alle in Ostdeutschland. Ich weise noch mal ausdrücklich darauf hin, weil wir in dieser Woche 30\xa0Jahre deutsche Einheit feiern; das wird am Freitag mit einer Debatte hier im Bundestag begleitet. Das Bundesumweltministerium hat in der letzten Woche daran erinnert, dass aus dem ehemaligen Todesstreifen, den die DDR an der Grenze zur Bundesrepublik errichtet hatte, in den vergangenen 30\xa0Jahren das Grüne Band geworden ist. Es ist inzwischen eine Lebenslinie, die Ost und West verbindet und heute zum Lebensraum von vielen bedrohten Tier- und Pflanzenarten und gleichzeitig ein sehr bemerkenswerter Ort der Erinnerungskultur geworden ist.'}, {'paragraphs': 'Umweltschutz wird auch hier immer wieder als Streitthema wahrgenommen. Das ist gut so; denn wir wollen den Wettstreit um die besten politischen Konzepte und Ideen. Was aber unterschätzt wird, ist, wie verbindend die Wirkung des Umweltschutzes ist. Das Grüne Band ist wirklich ein Symbol dafür. Wir verdanken der Umweltbewegung in der DDR einen wichtigen Anteil an der Friedlichen Revolution und auch das großartige Nationalparkprogramm, das inzwischen zu einer verbindenden Erfolgsgeschichte in ganz Deutschland geworden ist.'}, {'paragraphs': 'Meine Damen und Herren, ein Ende der weltweiten Coronapandemie ist noch nicht in Sicht. Wichtig ist, dass der wirtschaftliche Neustart nach dem Lockdown jetzt genutzt werden muss, um mit dem staatlichen Konjunkturprogramm den Aufbruch in eine klimaneutrale Zukunft zu beschleunigen, um den Weg dahin zu ebnen. Die Fenster, die Türen für diese Veränderungen stehen weit offen, und deswegen ist es jetzt so wichtig, zu handeln und umzusteuern.'}, {'paragraphs': 'Unsere deutsche Ratspräsidentschaft ist die perfekte Gelegenheit dafür, das europäische Band im Klimaschutz enger zu knüpfen. Der European Green Deal ist die richtige Antwort der EU-Kommission auf die Klimakrise genauso wie der Vorschlag für eine Anhebung des EU-Klimaziels auf mindestens 55\xa0Prozent Minderung der Treibhausgase. Beides liegt im deutschen Interesse und verdient unsere volle Unterstützung. Morgen beim informellen Treffen der EU-Umweltministerinnen und ‑minister werde ich mit daran arbeiten, dass wir bald eine Einigung auf europäischer Ebene bei der neuen Klimapolitik und den Klimazielen hinbekommen.'}, {'paragraphs': 'Meine Damen und Herren, schließen möchte ich mit einem Thema, das wichtig ist und das uns seit Anfang der Woche auch öffentlich stärker beschäftigt. Es geht darum, Verantwortung für die Endlagerung unseres Atommülls zu übernehmen. Drei Generationen haben die Atomkraft in Deutschland genutzt, 30\u202f000\xa0Generationen werden sich mit den Hinterlassenschaften beschäftigen müssen.'}, {'paragraphs': 'Gestern hat die Bundesgesellschaft für Endlagerung ihren ersten Zwischenbericht vorgelegt. Das ist ein ganz wichtiger Schritt auf dem Weg hin zu einer sicheren Lagerung der Hinterlassenschaften der Atomenergienutzung. Wir brauchen den sichersten Ort in Deutschland für den Atommüll. Entscheidend ist für mich\xa0– und ich denke, für alle hier im Haus\xa0–, dass das Verfahren strikt wissenschaftlich ist und dass die Geologie am Ende über den Standort entscheidet. Politische Überzeugungen dürfen hier keine Rolle spielen. Es geht nicht darum, das hier politisch auszufechten, sondern wir brauchen wirklich den sichersten Ort. Wir haben, glaube ich, über Jahrzehnte schmerzhaft gelernt, dass die Endlagersuche nur gemeinsam, dass sie nur solidarisch gelingen kann. Deswegen will ich die Gelegenheit nutzen, hier im Bundestag daran zu erinnern, dass wir mit allen 16\xa0Bundesländern, mit allen damals im Parlament vertretenen Parteien über Parteigrenzen hinweg diesen Weg der Suche beschlossen haben. Das ist ein breiter Konsens gewesen, und ich bitte darum, diesen Konsens zu bewahren. Ich erwarte von allen, dass sie zu dieser Verantwortung stehen.'}, {'paragraphs': 'Der Einzelplan\xa016 stellt sich den aktuellen und den langfristigen Herausforderungen. Ich freue mich sehr auf die Beratungen mit Ihnen und werbe natürlich um Ihre Unterstützung.'}, {'paragraphs': 'Herzlichen Dank.'}, {'paragraphs': 'Das Wort hat der Abgeordnete Karsten Hilse für die AfD-Fraktion.'}]

In [30]:
def create_paragraphs_classified_table(reset_db:bool=False):
    """
    Creates a table for classified paragraphs in the database.

    Args:
        reset_db (bool): If True, drops the table if it exists before creating it.
                         Defaults to False.
    Returns:
        None
    """
    if reset_db:
        con.execute("DROP TABLE IF EXISTS classified_paragraphs")
        con.execute("DROP SEQUENCE IF EXISTS classified_paragraphs_id_seq")

    # Create a sequence for the primary key
    con.execute("CREATE SEQUENCE IF NOT EXISTS classified_paragraphs_id_seq START 1;")

    con.execute("""
        CREATE TABLE IF NOT EXISTS classified_paragraphs (
            id INTEGER DEFAULT nextval('classified_paragraphs_id_seq') PRIMARY KEY,
            paragraph_id INTEGER, -- If its 0, its the first paragraph of the speech, 1 for the second, etc.
            speech_id INTEGER NOT NULL REFERENCES speech(id),
            paragraph VARCHAR NOT NULL,
            group_mention VARCHAR NOT NULL, -- This is the group mention, e.g. die Mitglieder der SPD-Fraktion
            label VARCHAR(15) NOT NULL,
        )
    """)
    con.commit()

In [43]:
def smart_join(tokens):
    """
    Joins a list of word tokens into a single string, handling punctuation
    and sub-word prefixes ('##') correctly.

    Args:
        tokens (list[str]): A list of word tokens, which may include sub-word tokens prefixed with '##'.

    Returns:
        str: A single string with tokens joined together, ensuring proper spacing around punctuation.
    """
    result = []
    # Define punctuation that should not have a preceding space
    no_space_before = {',', '.', '?', '!', ';', ':', ')'}
    
    for i, token in enumerate(tokens):
        # If it's the very first token, a punctuation mark, or a sub-word,
        # don't add a leading space.
        if i > 0 and token not in no_space_before and not token.startswith('##'):
            result.append(' ')
            
        # Append the token itself, removing any '##' prefixes
        result.append(token.replace('##', ''))
        
    return "".join(result).replace(' - ', '-') # Removes the space before and after a hyphen (Bindestrich)

def extract_groups(paragraph:list[tuple[str,str]]) -> list[tuple[str, str]]:
    """ Extracts group mention along with their labels from a paragraph. It groups tokens by their entity labels to get the full mention.
    If a mention is broken e.g it does not start with a 'B-' label, it will be filtered.

    Args:
        paragraph (list[tuple[str,str]]): A list of tuples containing tokens and their corresponding labels.

    Returns:
        list[tuple[str, str]]: A list of tuples where each tuple contains the entity label and a list of (token, label) pairs for that entity, which contain the full mention.
    """
    # This is a set of special tokens that should be ignored in the grouping process. -> adjust it if necessary
    SPECIAL = {"[CLS]", "[SEP]", "[PAD]", "[UNK]"}
    # empty list for groups of paragraph
    groups = []
    # This is a temporary list to hold the current group mention
    group_tmp = []
    # This is a flag to indicate if we are currently inside a group mention
    group_started = False
    entity = "" # This hold the current entity. e.g. EOPOL for B-EOPOL
    for token, label in paragraph:
        # If token is a special token like [CLS] skip it
        if token in SPECIAL:
            continue
        # Check for begin of group
        elif label.startswith("B-"):
            group_started = True
            entity = label[2:]
            if group_tmp:
                groups.append((entity, group_tmp))
                group_tmp = [] # New Group will begin
            # Append new beginning label
            group_tmp.append((token, label))
        # It is checked that 1) we have an inside label and 2) There was a B- label before!
        elif label.startswith("I-") and group_started:  
            # Then we check if the entity matches
            if label[2:] != entity:
                print(f"Current label: {label[2:]} doesn't match beginning label: {entity}")
                # Break current group because of the miss-label
                group_started = False
            else:
            # If all tests hold, we append the token and its label to the current group
                group_tmp.append((token, label))
        elif label == 'O':
            # An 'O' Tag is always outside. Thus if we scan one, it means that the current group is over
            if group_tmp:
                groups.append((entity, group_tmp))
                group_tmp = [] # New Group will begin
            group_started = False
        else:
            print(f"Filtered faulty classification: ({token}, {label})")
    
    # Flush last word
    if group_tmp:
        groups.append((entity, group_tmp))

    return groups

def insert_paragraph(speech_id:int, index:int, entity:str, group_clean_text:str, paragraph:str):
    """
    Inserts a classified paragraph into the database.

    Args:
        speech_id (int): The ID of the speech.
        index (int): The index of the paragraph in the speech. 0 for the first paragraph, 1 for the second, etc.
        entity (str): The entity label of the paragraph. For example, 'EOPOL' for B-EOPOL.
        group_clean_text (str): The cleaned text of the paragraph.
        paragraph (str): The original paragraph text.

    Returns:
        None
    """
    con.execute("""
        INSERT INTO classified_paragraphs (paragraph_id, speech_id, paragraph, group_mention, label)
        VALUES (?, ?, ?, ?, ?)
        ON CONFLICT DO NOTHING; -- If the paragraph already exists, do nothing
    """, (index, speech_id, paragraph, group_clean_text, entity))
    con.commit()

def insert_classified_paragraphs(speech_id:str, index:int, groups:list[tuple[str,list[tuple[str,str]]]], paragraph:str):
    """Inserts classified paragraphs into the database.

    Args:
        speech_id (str): The ID of the speech.
        index (int): The index of the paragraph in the speech. 0 for the first paragraph, 1 for the second, etc.
        groups (list[tuple[str,list[tuple[str,str]]]]): A list of tuples containing the entity and a list of the token, label pairs for each group.
        paragraph (str): The original paragraph text.

    Returns:
        None
    """
    for group in groups:
        entity, raw_tokens = group
        tokens = [item[0] for item in raw_tokens]
        group_clean_text = smart_join(tokens)
        print(f"{entity} -> {group_clean_text}")
        insert_paragraph(speech_id, index, entity, group_clean_text, paragraph)


def process_speech(speech_id:str, paragraphs:list[dict[str, list[str]]]):
    """Processes a speech by classifying its paragraphs (extracting group mention) and inserting them into the database.

    Args:
        speech_id (str): The ID of the speech.
        paragraphs (list[dict[str, list[str]]]): The list of paragraphs, each represented as a dictionary with a 'paragraphs' key containing the text.
    """
    classified_paragraphs = predict_batch(paragraphs)
    for index, p in enumerate(classified_paragraphs):
        # print(p)
        groups = extract_groups(index, p)
        insert_classified_paragraphs(speech_id, index, groups, paragraphs[index].get('paragraphs'))


def main():
    create_paragraphs_classified_table(reset_db=True)
    speech_id, paragraphs = preprocess_speech(test_speech)
    process_speech(speech_id, paragraphs)

if __name__ == "__main__":
    main()

Map: 100%|██████████| 16/16 [00:00<00:00, 5259.32 examples/s]


Batch size: 16
EPPOL -> Frau Präsidentin
EPPOL -> unser Finanzminister Olaf Scholz
EOPOL -> die Bundesregierung
EOPOL -> den Deutschen Bundestag
PGEN -> aller Menschen in unserem Land
EPPOL -> der Umweltministerin
EOPOL -> die gesamte Bundesregierung, also alle Ministerien
EOPOL -> alle Ministerien
EOPOL -> des Verkehrsministeriums
EPPOL -> Olaf Scholz
EOPOL -> des Wirtschafts -, des Landwirtschafts -
EOPOL -> des Bauministeriums
EOPOL -> Die Bundesregierung
EOPOL -> der Bundesrepublik
EOPOL -> die Bundesregierung
EOPOL -> mein Ministerium
GPE -> Deutschland
EONGO -> K
Filtered faulty classification: (Bio, I-EOPOL)
Filtered faulty classification: (##di, I-EOPOL)
Filtered faulty classification: (##vers, I-EOPOL)
Filtered faulty classification: (##mon, I-EOPOL)
Filtered faulty classification: (##itor, I-EOPOL)
Filtered faulty classification: (Leipzig, I-GPE)
EOPOL -> das Bundesumweltministerium
GPE -> der Lausitz
Filtered faulty classification: (Bundesrepublik, I-GPE)
GPE -> Ostdeutschla

In [35]:
process_speech(classified_paragraphs)
con.sql('create table speech (id integer primary key)')

[('[CLS]', 'O'), ('Frau', 'B-EPPOL'), ('Präsidentin', 'I-EPPOL'), ('!', 'O'), ('Meine', 'O'), ('sehr', 'O'), ('geehrt', 'O'), ('##en', 'O'), ('Damen', 'O'), ('und', 'O'), ('Herren', 'O'), ('Abgeordnete', 'O'), ('!', 'O'), ('Umwelt', 'O'), ('-', 'O'), ('und', 'O'), ('Klimaschutz', 'O'), ('zahlen', 'O'), ('sich', 'O'), ('aus', 'O'), ('[UNK]', 'O'), ('das', 'O'), ('hat', 'O'), ('heute', 'O'), ('Morgen', 'O'), ('unser', 'B-EPPOL'), ('Finanzminister', 'I-EPPOL'), ('Olaf', 'I-EPPOL'), ('Scho', 'I-EPPOL'), ('##l', 'I-EPPOL'), ('##z', 'I-EPPOL'), ('sehr', 'O'), ('eindrucks', 'O'), ('##voll', 'O'), ('dargelegt', 'O'), ('.', 'O'), ('Das', 'O'), ('ist', 'O'), ('eine', 'O'), ('der', 'O'), ('zentralen', 'O'), ('Botschaft', 'O'), ('##en', 'O'), ('des', 'O'), ('Bundes', 'O'), ('##haushalt', 'O'), ('##es', 'O'), ('für', 'O'), ('20', 'O'), ('##21', 'O'), (',', 'O'), ('den', 'O'), ('die', 'B-EOPOL'), ('Bundesregierung', 'I-EOPOL'), ('diese', 'O'), ('Woche', 'O'), ('hier', 'O'), ('in', 'O'), ('den', 'B-E

In [13]:
print(classified_paragraphs[8])

[('[CLS]', 'O'), ('Umweltschutz', 'O'), ('wird', 'O'), ('auch', 'O'), ('hier', 'O'), ('immer', 'O'), ('wieder', 'O'), ('als', 'O'), ('Streit', 'O'), ('##the', 'O'), ('##ma', 'O'), ('wahrgenommen', 'O'), ('.', 'O'), ('Das', 'O'), ('ist', 'O'), ('gut', 'O'), ('so', 'O'), (';', 'O'), ('denn', 'O'), ('wir', 'O'), ('wollen', 'O'), ('den', 'O'), ('Wett', 'O'), ('##streit', 'O'), ('um', 'O'), ('die', 'O'), ('besten', 'O'), ('politischen', 'O'), ('Konzepte', 'O'), ('und', 'O'), ('Ideen', 'O'), ('.', 'O'), ('Was', 'O'), ('aber', 'O'), ('untersch', 'O'), ('##ätzt', 'O'), ('wird', 'O'), (',', 'O'), ('ist', 'O'), (',', 'O'), ('wie', 'O'), ('verbinden', 'O'), ('##d', 'O'), ('die', 'O'), ('Wirkung', 'O'), ('des', 'O'), ('Umweltschutz', 'O'), ('##es', 'O'), ('ist', 'O'), ('.', 'O'), ('Das', 'B-EOPOL'), ('Grüne', 'I-EOPOL'), ('Band', 'I-EOPOL'), ('ist', 'O'), ('wirklich', 'O'), ('ein', 'O'), ('Symbol', 'O'), ('dafür', 'O'), ('.', 'O'), ('Wir', 'O'), ('verdanken', 'O'), ('der', 'O'), ('Umwelt', 'I-EOMO